Current tasks
-------------
- Set _eval_energy and eval_energy to be nogil
- ~~Make CompiledModel a cdef class~~
- Link CompiledModel-based energy evaluation into the main solver
- Decide how/if to store gradients and Hessians of RedlichKisterSums
- Write analagous eval_gradient and eval_hessian functions
- Link gradient and Hessian for CompiledModel into main solver

In [1]:
from pycalphad.core.compiled_model import CompiledModel
from pycalphad import calculate, Database, Model
from pycalphad.core.phase_rec import PhaseRecord
import pycalphad.variables as v
import numpy as np

In [2]:
dbf = Database('alcocr-sandbox.tdb')

In [3]:
cmpmdl = CompiledModel(dbf, ['AL', 'CO', 'CR', 'VA'], 'BCC_A2')
out = np.zeros(7)
cmpmdl.eval_energy_gradient(out, np.array([101325, 600, 0.3, 0.3, 0.4-1e-12, 1e-12, 1]), np.array([]))
out

array([  0.00000000e+00,  -9.05365885e+00,  -1.01755913e+03,
        -1.01755913e+03,   4.17600426e+02,  -1.38286370e+05,
        -1.33048592e+03])

In [4]:
mdl = Model(dbf, ['AL', 'CO', 'CR', 'VA'], 'BCC_A2')
print(mdl.GM.diff(v.Y('BCC_A2', 1, 'VA')))
[mdl.GM.diff(x).subs({v.T: 600., v.P:101325., v.Y('BCC_A2', 0, 'AL'): 0.3, v.Y('BCC_A2', 0, 'CO'): 0.3,
                    v.Y('BCC_A2', 0, 'CR'): 0.4-1e-12, v.Y('BCC_A2', 0, 'VA'): 1e-12, v.Y('BCC_A2', 1, 'VA'): 1.})
 for x in [v.P, v.T, v.Y('BCC_A2', 0, 'AL'),v.Y('BCC_A2', 0, 'CO'),v.Y('BCC_A2', 0, 'CR'),v.Y('BCC_A2', 0, 'VA'), v.Y('BCC_A2', 1, 'VA')]]

8.3145*T*(3.0*Piecewise((log(BCC_A21VA) + 1, BCC_A21VA > 1.0e-13), (0, True))/(-1.0*BCC_A20VA - 3.0*BCC_A21VA + 4.0) + 3.0*Piecewise((BCC_A20AL*log(BCC_A20AL), BCC_A20AL > 1.0e-13), (0, True))/(-1.0*BCC_A20VA - 3.0*BCC_A21VA + 4.0)**2 + 3.0*Piecewise((BCC_A20CO*log(BCC_A20CO), BCC_A20CO > 1.0e-13), (0, True))/(-1.0*BCC_A20VA - 3.0*BCC_A21VA + 4.0)**2 + 3.0*Piecewise((BCC_A20CR*log(BCC_A20CR), BCC_A20CR > 1.0e-13), (0, True))/(-1.0*BCC_A20VA - 3.0*BCC_A21VA + 4.0)**2 + 3.0*Piecewise((BCC_A20VA*log(BCC_A20VA), BCC_A20VA > 1.0e-13), (0, True))/(-1.0*BCC_A20VA - 3.0*BCC_A21VA + 4.0)**2 + 9.0*Piecewise((BCC_A21VA*log(BCC_A21VA), BCC_A21VA > 1.0e-13), (0, True))/(-1.0*BCC_A20VA - 3.0*BCC_A21VA + 4.0)**2)


[0,
 -9.05365884524735,
 -1017.55912894182,
 -1017.55912894182,
 417.600425887349,
 -138286.370348319,
 -1330.48592144656]

In [ ]:
%time res = calculate(dbf, ['AL', 'CO', 'CR', 'VA'], sorted(dbf.phases.keys()), T=2000, P=101325, output='GM', pdens=200, model=CompiledModel)
res.GM[..., :20]

In [ ]:
%time res2 = calculate(dbf, ['AL', 'CO', 'CR', 'VA'], sorted(dbf.phases.keys()), T=2000, P=101325, output='GM', pdens=200, model=Model)
res2.GM[..., :20]